In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the dataset from the specified path
df = pd.read_csv('/content/drive/MyDrive/ML project/fake_real.csv', sep=',', encoding='utf-8', quotechar='"')


In [ ]:
# Display the first few rows of the DataFrame
df.head()

,title,text,subject,date,type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True


In [ ]:
# Drop all columns except 'text' and 'type'
df = df[['text', 'type']]

# Verify the resulting DataFrame
df.head()

,text,type
0,WASHINGTON (Reuters) - The head of a conservat...,True
1,WASHINGTON (Reuters) - Transgender people will...,True
2,WASHINGTON (Reuters) - The special counsel inv...,True
3,WASHINGTON (Reuters) - Trump campaign adviser ...,True
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,True


**Text processing function using NLTK** 3.34

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re

# Download NLTK resources if not already present
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Text processing function using NLTK
def preprocess_nltk(text):
    # Tokenization
    tokens = word_tokenize(text)
    # Remove punctuation marks, special characters, and digits
    tokens = [token for token in tokens if token not in string.punctuation and not token.isdigit()]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Apply text processing to the 'text' column
df['Cleaned_Text'] = df['text'].apply(preprocess_nltk)

# Display processed 'text' column
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,type,Cleaned_Text
0,WASHINGTON (Reuters) - The head of a conservat...,True,WASHINGTON Reuters head conservative Republica...
1,WASHINGTON (Reuters) - Transgender people will...,True,WASHINGTON Reuters Transgender people allowed ...
2,WASHINGTON (Reuters) - The special counsel inv...,True,WASHINGTON Reuters special counsel investigati...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,True,WASHINGTON Reuters Trump campaign adviser Geor...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,True,SEATTLE/WASHINGTON Reuters President Donald Tr...


**Text Processing with Gensim** 45s
* Gensim, a library for topic modeling and similarity detection, lacks built-in functionality for all text preprocessing tasks. It only provides tokenization, but not punctuation marks, special characters, numbers/digits, stopwords removal, lemmatization, or joining tokens back into a single string. Additional libraries are needed for more advanced tasks.

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.porter import PorterStemmer

# Text processing function using Gensim
def preprocess_gensim(text):
    # Tokenization and remove stopwords
    tokens = [token for token in simple_preprocess(text) if token not in STOPWORDS]
    # Remove punctuation marks, special characters, and digits
    tokens = [re.sub(r'\W+', '', token) for token in tokens if not token.isdigit()]
    # Lemmatization
    return ' '.join(tokens)
# Apply text processing to the 'text' column
df['Cleaned_Text'] = df['text'].apply(preprocess_gensim)

# Display processed 'text' column
df.head()

,text,type,Cleaned_Text
0,WASHINGTON (Reuters) - The head of a conservat...,True,washington reuters head conservative republica...
1,WASHINGTON (Reuters) - Transgender people will...,True,washington reuters transgender people allowed ...
2,WASHINGTON (Reuters) - The special counsel inv...,True,washington reuters special counsel investigati...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,True,washington reuters trump campaign adviser geor...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,True,seattle washington reuters president donald tr...


**Text Processing with spaCy** 1 hr
* "en_core_web_sm" is a smaller,while "en_core_web_lg" is a larger

In [ ]:
import spacy
import re
import string

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Function to remove noise from text
    def remove_noise(text):
        # Remove punctuation marks, special characters, and digits in one pass
        cleaned_text = re.sub(r'[^\w\s]|[\d]', '', text)
        return cleaned_text

    # Function to normalize text (handle contractions, convert to lowercase)
    def normalize_text(text):
        # Convert text to lowercase
        text = text.lower()
        # Handle contractions (e.g., "can't" -> "can not")
        text = re.sub(r"n't", " not", text)
        # Add more contraction handling patterns as needed
        return text

    # Function to tokenize the text using spaCy
    def tokenize_text(text):
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_digit]
        return tokens

    # Function to remove stopwords from tokens
    def remove_stopwords(tokens):
        # Define stop words
        stop_words = set(nlp.Defaults.stop_words)
        return [token for token in tokens if token not in stop_words]

    # Function to join tokens back into a single string
    def join_tokens(tokens):
        return ' '.join(tokens)

    # Remove noise from the text
    text = remove_noise(text)
    # Normalize the text (handle contractions, convert to lowercase)
    text = normalize_text(text)
    # Tokenize the text using spaCy and remove punctuation marks, spaces, and digits
    tokens = tokenize_text(text)
    # Remove stopwords from tokens
    tokens = remove_stopwords(tokens)
    # Join tokens back into a single string
    processed_text = join_tokens(tokens)

    return processed_text

# Apply the preprocess_text function to the 'text' column
df['processed_Text'] = df['text'].apply(preprocess_text)

# Display the DataFrame after text processing
df.head()

,text,type,Cleaned_Text
0,WASHINGTON (Reuters) - The head of a conservat...,True,WASHINGTON Reuters the head of a conservative ...
1,WASHINGTON (Reuters) - Transgender people will...,True,WASHINGTON Reuters Transgender people will be ...
2,WASHINGTON (Reuters) - The special counsel inv...,True,WASHINGTON Reuters the special counsel investi...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,True,WASHINGTON Reuters trump campaign adviser Geor...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,True,SEATTLE WASHINGTON Reuters President Donald Tr...


1. **In general**, while **spaCy** may have slightly longer processing times then **nltk** due to its more comprehensive functionality and language model loading
2. If we prioritize speed, efficiency, and accuracy in text processing, spaCy may be the better choice.
3. However, if we require more flexibility, customization, and a wide range of NLP tools, **NLTK** might be a better fit.

In [ ]:
import re
import string
import nltk
import spacy

# Download NLTK resources if not already present
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize NLTK components
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess_text(text):
    # Tokenization using spaCy
    doc = nlp(text)
    tokens_spacy = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space and not token.is_digit]

    # Tokenization and remove stopwords using NLTK
    tokens_nltk = nltk.word_tokenize(text.lower())
    tokens_nltk = [token for token in tokens_nltk if token not in stop_words and token not in string.punctuation]

    # Combine tokens from both NLTK and spaCy
    tokens_combined = tokens_spacy + tokens_nltk

    # Lemmatization using NLTK
    tokens_combined = [lemmatizer.lemmatize(token) for token in tokens_combined]

    # Join tokens back into a single string
    processed_text = ' '.join(tokens_combined)

    return processed_text

# Apply text processing to the 'text' column
df['Cleaned_Text'] = df['text'].apply(preprocess_text)

# Display processed 'text' column
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,type,Cleaned_Text
0,WASHINGTON (Reuters) - The head of a conservat...,True,WASHINGTON Reuters head conservative Republica...
1,WASHINGTON (Reuters) - Transgender people will...,True,WASHINGTON Reuters Transgender people allowed ...
2,WASHINGTON (Reuters) - The special counsel inv...,True,WASHINGTON Reuters special counsel investigati...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,True,WASHINGTON Reuters Trump campaign adviser Geor...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,True,SEATTLE/WASHINGTON Reuters President Donald Tr...


In [ ]:
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources if not already present
nltk.download('punkt')
nltk.download('stopwords')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def process_text(text):
    # Function to remove punctuation marks from text
    def remove_punctuation(text):
        punctuation_pattern = r'[^\w\s]'
        cleaned_text = re.sub(punctuation_pattern, '', text)
        return cleaned_text

    # Function to remove special characters and numbers from text
    def remove_special_characters_and_numbers(text):
        special_characters_and_numbers_pattern = r'[^a-zA-Z\s]'
        cleaned_text = re.sub(special_characters_and_numbers_pattern, '', text)
        return cleaned_text

    # Function to remove digits from text
    def remove_digits(text):
        digit_pattern = r'\d+'
        cleaned_text = re.sub(digit_pattern, '', text)
        return cleaned_text

    # Function to convert text to lowercase
    def convert_to_lowercase(text):
        return text.lower()

    # Function to remove stopwords from tokens using NLTK
    def remove_stopwords(tokens):
        stop_words = set(stopwords.words('english'))
        return [token for token in tokens if token not in stop_words]

    # Function to join tokens back into a single string
    def join_tokens(tokens):
        return ' '.join(tokens)

    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    tokens_spacy = [token.lemma_ for token in doc]

    cleaned_text = remove_punctuation(text)
    cleaned_text = remove_special_characters_and_numbers(cleaned_text)
    cleaned_text = remove_digits(cleaned_text)  # Remove digits
    cleaned_text = convert_to_lowercase(cleaned_text)
    tokens = word_tokenize(cleaned_text)  # Tokenization using NLTK
    tokens = remove_stopwords(tokens)
    tokens.extend(tokens_spacy)  # Add spaCy tokens
    processed_text = join_tokens(tokens)

    return processed_text

# Apply the process_text function to the 'text' column of DataFrame 2
df['Processed_Text'] = df['text'].apply(process_text)

# Display the DataFrame after text preprocessing
df.head()

In this function:

1. We use spaCy for tokenization and lemmatization to take advantage of its efficiency and accuracy.
2. We use NLTK for additional tokenization, stopwords removal, and lemmatization.
3. We combine tokens from both NLTK and spaCy to leverage the strengths of both libraries.
4. Finally, we join the processed tokens back into a single string.